# **1. Install PyTorch Lightning**

> **Pip users**: pip install lightning
>
> **Conda users**: conda install lightning -c conda-forge

# **2. Define a LightningModule**

A LightningModule enables your PyTorch nn.Module to play together in complex ways inside the training_step (there is also an optional validation_step and test_step).

In [10]:
import os
import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch

        # returns a new tensor with the same data as the self tensor but of a different shape.
        x = x.view(x.size(0), -1) # equivalent to nn.Flatten()
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)

        # logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

# **3. Define a dataset**

Lightning supports ANY iterable (DataLoader, numpy, etc…) for the train/val/test/predict splits.

In [3]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)  # utils (from torch)

# **4. Train the model**

The Lightning Trainer “mixes” any LightningModule with any dataset and abstracts away all the engineering complexity needed for scale.

The Lightning Trainer automates 40+ tricks including:

* Epoch and batch iteration

* `optimizer.step()`, loss.backward(), optimizer.zero_grad() calls

* Calling of `model.eval()`, enabling/disabling grads during evaluation

* Checkpoint Saving and Loading

* Tensorboard (see loggers options)

* Multi-GPU support

* TPU

* 16-bit precision AMP support

>
> **Trainer**: https://lightning.ai/docs/pytorch/stable/common/trainer.html
>
> **Trainer tricks**: https://lightning.ai/docs/pytorch/stable/common/trainer.html#trainer-flags
>

In [8]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Adombi\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_ma

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


# **5. Use the model**

Once you’ve trained the model you can export to onnx, torchscript and put it into production or simply load the weights and run predictions.

In [11]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = torch.rand(4, 28 * 28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 0.3383, -0.0650,  0.3435],
        [ 0.2860, -0.0427,  0.4032],
        [ 0.2403,  0.0127,  0.3409],
        [ 0.1840, -0.0436,  0.3422]], device='cuda:0',
       grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡


# **6. Visualize training**

> Install tensorboard
> * Open CMD and `pip install tensorboard`
>
> Install torch-tb-profiler
> * Open CMD and `pip install torch-tb-profiler`
>
> Run tensorboard
>
> * ``

If you have tensorboard installed, you can use it for visualizing experiments. Run this on your commandline and open your browser to http://localhost:6006/


**Use tensorboard with PyTorch**

> https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# **7. Supercharge training**

Enable advanced training features using Trainer arguments. These are state-of-the-art techniques that are automatically integrated into your training loop without changes to your code.

In [ ]:
# train on 4 GPUs
trainer = L.Trainer(
    devices=4,
    accelerator="gpu",
 )

# train 1TB+ parameter models with Deepspeed/fsdp
trainer = L.Trainer(
    devices=4,
    accelerator="gpu",
    strategy="deepspeed_stage_2",
    precision=16
 )

# 20+ helpful flags for rapid idea iteration
trainer = L.Trainer(
    max_epochs=10,
    min_epochs=5,
    overfit_batches=1
 )

# access the latest state of the art techniques
trainer = L.Trainer(callbacks=[StochasticWeightAveraging(...)])

# **8. Maximize flexibility**

Lightning’s core guiding principle is to always provide maximal flexibility without ever hiding any of the PyTorch.

Lightning offers 5 added degrees of flexibility depending on your project’s complexity.

## **8.1. Customize training loop**

https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#lightning-hooks

In [ ]:
class LitAutoEncoder(L.LightningModule):
    def backward(self, loss):
        loss.backward()

## **8.2. Extend the Trainer**

If you have multiple lines of code with similar functionalities, you can use callbacks to easily group them together and toggle all of those lines on or off at the same time.

In [ ]:
trainer = Trainer(callbacks=[AWSCheckpoints()])

## **8.3. Use a raw PyTorch loop**

For certain types of work at the bleeding-edge of research, Lightning offers experts full control of optimization or the training loop in various ways.

https://lightning.ai/docs/pytorch/stable/model/build_model_advanced.html#manual-optimization